In [1]:
import pandas as pd
import os
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display

# ตั้งค่า Path
file_name = r"d:\year4\สหกิจ\prachatai_test.csv"

# โหลด Stopwords
stop_words = set(thai_stopwords())
my_custom_stops = {
    ' ', '\n', '\t', '“', '”', '(', ')', '[', ']', 
    '-', '.', ',', ':', '/', '?', '!',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
}
stop_words.update(my_custom_stops)

print("✅ Setup & Stopwords Ready.")

✅ Setup & Stopwords Ready.


In [2]:
# ฟังก์ชันทำความสะอาดข้อมูล
def text_process(text):
    tokens = word_tokenize(str(text), engine='newmm')
    clean_tokens = []
    for w in tokens:
        if w not in stop_words and w.strip() != '':
            clean_tokens.append(w)
    
    # รวมกลับเป็นสตริงคั่นด้วยช่องว่าง 
    # (เพราะ TfidfVectorizer ต้องการ input เป็น string ยาวๆ ที่มีเว้นวรรค)
    return ' '.join(clean_tokens)

# โหลดข้อมูล
if os.path.exists(file_name):
    df = pd.read_csv(file_name)
    # เลือกมาแค่ 5 ข่าวแรกเพื่อทดสอบ
    df_small = df.head(5).copy()
    print(f"✅ Loaded {len(df_small)} rows for testing.")
else:
    print(f"❌ Error: File not found at {file_name}")

✅ Loaded 5 rows for testing.


In [3]:
print("🔄 กำลังทำความสะอาดข้อมูล (Tokenizing & Cleaning)...")
df_small['clean_text'] = df_small['body_text'].apply(text_process)

print("✅ Done!")
print("\n--- 🔍 เปรียบเทียบข้อมูล ---")
display(df_small[['body_text', 'clean_text']].head(2))

🔄 กำลังทำความสะอาดข้อมูล (Tokenizing & Cleaning)...
✅ Done!

--- 🔍 เปรียบเทียบข้อมูล ---


,body_text,clean_text
0,17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเก...,17 พ ย 2558 Blognone [1] รายงาน แฮคเกอร์ Anony...
1,ขณะที่ ส.ส. ญี่ปุ่นต้องการแก้รัฐธรรมนูญเพื่อเป...,ส.ส. ญี่ปุ่น ต้องการ แก้ รัฐธรรมนูญ กองกำลัง ป...


In [4]:
# สร้างตัวแปลง TF-IDF
# tokenizer=lambda x: x.split() -> บอกมันว่า "ไม่ต้องตัดคำแล้วนะ ฉันตัดมาให้แล้ว แค่แยกตามช่องว่างพอ"
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(), token_pattern=None)

# แปลงข้อความเป็น Matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df_small['clean_text'])

print("--- 📊 ผลลัพธ์ Feature Extraction ---")
print(f"ขนาดของ Matrix (จำนวนข่าว, จำนวนศัพท์ที่ไม่ซ้ำกัน): {tfidf_matrix.shape}")

--- 📊 ผลลัพธ์ Feature Extraction ---
ขนาดของ Matrix (จำนวนข่าว, จำนวนศัพท์ที่ไม่ซ้ำกัน): (5, 735)


In [5]:
# ===========================
# 🔍 เลือกดูข่าวลำดับที่เท่าไหร่ (0-4)
doc_idx = 0 
# ===========================

if doc_idx < len(df_small):
    # ดึงชื่อคอลัมน์ (คำศัพท์ทั้งหมด)
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # ดึงคะแนนของข่าวนั้นออกมา
    first_document_vector = tfidf_matrix[doc_idx]

    # สร้าง Dataframe เพื่อแสดงผลสวยๆ
    df_tfidf = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf_score"])
    
    # เรียงลำดับจากคะแนนมากไปน้อย
    top_keywords = df_tfidf.sort_values(by=["tfidf_score"], ascending=False).head(15)

    print(f"--- 🏆 15 คำเด่นประจำข่าวที่ {doc_idx+1} (Top Keywords) ---")
    print(f"หัวข้อข่าว: {df_small['title'].iloc[doc_idx]}")
    display(top_keywords)
else:
    print("❌ เลข Index เกินจำนวนข้อมูล")

--- 🏆 15 คำเด่นประจำข่าวที่ 1 (Top Keywords) ---
หัวข้อข่าว: แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ่สุดกับกลุ่ม IS


,tfidf_score
anonymous,0.514394
is,0.367424
โจมตี,0.296435
แฮคเกอร์,0.220454
ประกาศสงคราม,0.177861
กรุง,0.146970
บัญชี,0.146970
ปารีส,0.146970
สำนักพิมพ์,0.146970
charlie,0.146970
